In [1]:

import pandas as pd
import numpy as np
import time
from datetime import date, timedelta, datetime
import pymysql
from scipy.stats import norm

from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
### ssh -i "C:\Users\16784\Documents\ssh-key-2024-03-26.key" opc@150.136.59.147
### nohup dolt sql-server --host=0.0.0.0 --port=3306 --user=root --password=6qLwdLEUarSs23y > dolt_server.log 2>&1 &
### nohup dolt sql-server --host=0.0.0.0 --port=3307 --user=root --password=6qLwdLEUarSs23y > dolt_server.log 2>&1 &
### nohup dolt sql-server --host=0.0.0.0 --port=3308 --user=root --password=6qLwdLEUarSs23y > dolt_server.log 2>&1 &

In [3]:
username = 'root'
password = '6qLwdLEUarSs23y'
host = '150.136.59.147'
port_earnings = '3306' 
port_stocks = '3307' 
port_options = '3308' 

In [4]:
# Setting up connections to mysql servers running on free oracle vm
# although I also connected to the servers using the built-in sql support of datalore(sql cells are amazing)
# eventually I modified everything so this notebook can operate in other environments as well

connection_earnings = f'mysql+pymysql://{username}:{password}@{host}:{port_earnings}/earnings'
connection_stocks = f'mysql+pymysql://{username}:{password}@{host}:{port_stocks}/stocks'
connection_options = f'mysql+pymysql://{username}:{password}@{host}:{port_options}/options'
engine_e = create_engine(connection_earnings)
engine_s = create_engine(connection_stocks)
engine_o = create_engine(connection_options)

In [5]:
def take_ticker():
    """

    :rtype: object
    """
    while True:  # Keep asking until a valid ticker is entered
        input_ticker = input("Enter a stock ticker: ")
        ticker = input_ticker.upper()

        # Query to check if the ticker exists in the earnings_calendar table
        query = text("SELECT 1 FROM earnings_calendar WHERE act_symbol = :ticker LIMIT 1")
        # Execute the query and fetch the result
        with engine_e.connect() as connection:
            # Using parameters to safely insert the ticker into the query
            result = connection.execute(query, {"ticker": ticker})
            # Fetching the first row of the result
            exists_row = result.fetchone()
            # Since the query was only "SELECT 1", if the query returned a row - the ticker exists
            exists = exists_row is not None

        if exists:
            return ticker
        else:
            print(f"The ticker '{ticker}' is not found in the database. Please try again.")

In [6]:
def find_earnings_dates(ticker):
    query = text("""
    SELECT *
    FROM earnings_calendar
    WHERE act_symbol = :ticker
    ORDER BY date;
    """)
    
    with engine_e.connect() as connection:
        result = connection.execute(query, {"ticker": ticker})
        df_earnings_date = pd.DataFrame(result.fetchall(), columns=result.keys())

    df_earnings_date['date'] = pd.to_datetime(df_earnings_date['date']).dt.date
    condition = df_earnings_date['when'] == 'After market close'
    df_earnings_date.loc[condition, 'date'] = pd.to_datetime(df_earnings_date.loc[condition, 'date']) + pd.Timedelta(days=1)
    df_earnings_date['date'] = df_earnings_date['date'].apply(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
    
    return df_earnings_date

In [7]:
def fetch_stock_prices_around_earnings(ticker, earnings_dates, days_before=5, days_after=5):
    all_stock_prices = []  # List to hold data for all earnings dates
    period_counter = 1  # Initialize counter for the reporting period identifier
    last_earnings_date = None  # Track the last earnings date processed

    for earnings_date in earnings_dates:
        # Ensure each earnings_date is a datetime.date object for manipulation
        if isinstance(earnings_date, str):
            earnings_date = pd.to_datetime(earnings_date).date()

        # Since the data has some glitch, check if there are at least 60 days since the last earnings date processed
        if last_earnings_date and (earnings_date - last_earnings_date).days < 60:
            continue  # Skip this earnings date if the condition is not met

        # Calculate the start and end dates for the SQL query
        start_date = earnings_date - pd.Timedelta(days=days_before)
        end_date = earnings_date + pd.Timedelta(days=days_after)
        
        # Use text() to prepare the query
        query = text("""
        SELECT date, close, open, high, low
        FROM ohlcv
        WHERE act_symbol = :ticker
          AND date BETWEEN :start_date AND :end_date
        ORDER BY date;
        """)
        
        with engine_s.connect() as connection:
            # Execute the query with parameters
            result = connection.execute(query, {"ticker": ticker, "start_date": start_date, "end_date": end_date})
            # Fetch all results
            rows = result.fetchall()
            # If rows are fetched, create a DataFrame, add the period column, and append to the list
            if rows:
                df_stock_prices = pd.DataFrame(rows, columns=result.keys())
                df_stock_prices['period'] = period_counter  # Add the period identifier column
                all_stock_prices.append(df_stock_prices)
            
        last_earnings_date = earnings_date        
        period_counter += 1  # Increment the period counter after each iteration

    # Concatenate all DataFrames in the list into a single DataFrame
    if all_stock_prices:
        df_all_stock_prices = pd.concat(all_stock_prices, ignore_index=True)
    else:
        df_all_stock_prices = pd.DataFrame(columns=['date', 'close', 'open', 'high', 'low', 'period'])
    
    return df_all_stock_prices

In [24]:
def calculate_percentage_changes(df):
    # Ensure data is sorted by period and then by date to maintain order
    df = df.sort_values(by=['period', 'date'])
    df['close'] = df['close'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    # Calculate daily percentage change in closing prices within each period
    df['close_pct_change'] = df.groupby('period')['close'].pct_change() * 100
    
    period_extremes = df.groupby('period').agg(
        lowest_low=('low', 'min'), 
        highest_high=('high', 'max'),
        vix = ('close_pct_change', 'std')
    ).reset_index()

    # Step 2: Calculate the percentage difference between the lowest low and the highest high within each period
    period_extremes['largest_pct_diff'] = (
        (period_extremes['highest_high'] - period_extremes['lowest_low']) / period_extremes['lowest_low']
    ) * 100
    
    # Merge this back with the original dataframe to include the largest percentage difference for each period
    df = df.merge(period_extremes, on='period', how='left')
    
    # For each period, find the total percentage change
    first_last_close = df.groupby('period')['close'].agg(['first', 'last']).reset_index()
    first_last_close['total_close_pct_change'] = ((first_last_close['last'] - first_last_close['first']) / first_last_close['first']) * 100
    # Merge the total percentage change for each period back into the original DataFrame
    df = df.merge(first_last_close, on='period', how='left')
    
    return df

# Assuming df_all_stock_prices is DataFrame from fetch_stock_prices_around_earnings function
# processed_df = calculate_percentage_changes(df_all_stock_prices)
# print(processed_df)

In [56]:
## get the historical prices of a given stock and label with earnings dates
   
def fetch_prices_with_period(ticker, earnings_dates, days_before=1, days_after=7):
    period_counter = 1  
    last_earnings_date = None  

    if isinstance(earnings_dates, str):
            earnings_dates = pd.to_datetime(earnings_dates).date()
    # Calculate the start and end dates for the SQL query

    start_date = earnings_dates[0] - pd.Timedelta(days = 90)
    end_date = earnings_dates[len(earnings_dates) - 1] + pd.Timedelta(days = days_after)
    
    # Use text() to prepare the query
    query = text("""
    SELECT date, close, open, high, low
    FROM ohlcv
    WHERE act_symbol = :ticker
        AND date BETWEEN :start_date AND :end_date
    ORDER BY date;
    """)
    
    with engine_s.connect() as connection:
        result = connection.execute(query, {"ticker": ticker, "start_date": start_date, "end_date": end_date})
        rows = result.fetchall()
        if rows:
            df_stock_prices = pd.DataFrame(rows, columns=result.keys())

        
    df_stock_prices['period'] = 0
    df_stock_prices['earnings_release'] = 0

    for i, earnings_date in enumerate(earnings_dates):
        # Define the period range
        period_start = earnings_date - pd.Timedelta(days=days_before)
        period_end = earnings_date + pd.Timedelta(days=days_after)
        
        # Label the period number
        df_stock_prices.loc[(df_stock_prices['date'] >= period_start) & (df_stock_prices['date'] <= period_end), 'period'] = i + 1
        
        # Mark the earnings release window
        df_stock_prices.loc[(df_stock_prices['date'] >= period_start) & (df_stock_prices['date'] <= period_end), 'earnings_release'] = 1
        
        # Update period number for dates after the current earnings date until the next earnings date
        if i < len(earnings_dates) - 1:
            next_period_start = earnings_dates[i + 1] - pd.Timedelta(days=days_before)
            df_stock_prices.loc[(df_stock_prices['date'] > period_end) & (df_stock_prices['date'] < next_period_start), 'period'] = i + 1

    # Special handling for dates before the first period and after the last period
    df_stock_prices.loc[df_stock_prices['date'] < earnings_dates[0], 'period'] = 0  # First period starts from the beginning of the dataset
    df_stock_prices.loc[df_stock_prices['date'] > earnings_dates[len(earnings_dates) - 1], 'period'] += 1  # Increment period after the last earnings date

    return df_stock_prices


In [57]:
tk_0 = take_ticker()
df_date = find_earnings_dates(tk_0)
df_test = fetch_prices_with_period(tk_0, df_date['date'])

Enter a stock ticker:  meta


In [58]:
df_test

,date,close,open,high,low,period,earnings_release
0,2022-06-13,164.26,170.59,172.58,164.03,0,0
1,2022-06-14,163.73,166.03,166.75,161.36,0,0
2,2022-06-15,169.35,167.20,172.16,163.98,0,0
3,2022-06-16,160.87,163.72,165.08,159.61,0,0
4,2022-06-17,163.74,161.68,165.90,159.90,0,0
...,...,...,...,...,...,...,...
441,2024-03-19,496.24,488.17,496.63,481.28,7,0
442,2024-03-20,505.52,499.50,508.20,495.17,7,0
443,2024-03-21,507.76,514.71,515.04,506.01,7,0
444,2024-03-22,509.58,507.00,509.97,504.34,7,0


In [ ]:
# Calculate pct change, largest diff, volatility, etc. in and out reporting period

def 

In [9]:
## Wrong. Needs rework

def calculate_statistical_breakeven(df, call_price, call_strike, put_price, put_strike, current_price, percentile=0.5):
    total_premium = call_price + put_price
    upper_breakeven = call_strike + total_premium
    lower_breakeven = put_strike - total_premium
    
    # Using close percentage changes as a proxy for daily movement. Adjust as needed.
    df['daily_pct_change'] = df['close'].pct_change() * 100
    mean_pct_change = df['daily_pct_change'].mean()
    std_pct_change = df['daily_pct_change'].std()

    # Adjust the mean and std based on the specified percentile (for a more conservative estimate)
    if percentile < 0.5:
        # Focusing on smaller changes
        adjustment_factor = norm.ppf(percentile)
    else:
        # Focusing on larger changes, more optimistic
        adjustment_factor = norm.ppf(1 - percentile)

    adjusted_mean = mean_pct_change * adjustment_factor
    adjusted_std = std_pct_change * adjustment_factor

    # Calculate Z-scores for upper and lower breakeven points
    z_score_upper = (upper_breakeven - current_price - adjusted_mean) / adjusted_std
    z_score_lower = (current_price - lower_breakeven - adjusted_mean) / adjusted_std

    # Calculate probabilities using the cumulative distribution function
    probability_upper = norm.cdf(z_score_upper)
    probability_lower = norm.cdf(z_score_lower)

    # Combined chance to reach either breakeven point
    chance_of_breakeven = (probability_upper + (1 - probability_lower)) * 100 / 2

    return {
        "Upper Breakeven": upper_breakeven,
        "Lower Breakeven": lower_breakeven,
        "Chance of Breakeven (%)": chance_of_breakeven,
        "Adjusted Mean (%)": adjusted_mean,
        "Adjusted Std Deviation (%)": adjusted_std
    }





In [10]:
# Find all companies releasing their earnings in the next month
def fetch_earnings_next_month():
    today = datetime.now().date()
    next_month = today + timedelta(days=14)
    
    query = text("""
    SELECT act_symbol, date, `when`
    FROM earnings_calendar
    WHERE date BETWEEN :today AND :next_month
    ORDER BY date;
    """)

    with engine_e.connect() as connection:
        earnings_next_month = pd.read_sql_query(query, connection, params={"today": today, "next_month": next_month})
    
    return earnings_next_month



In [12]:
earnings_release_next_month = fetch_earnings_next_month()
print(earnings_release_next_month)

    act_symbol        date                when
0          AGX  2024-04-10                None
1         APOG  2024-04-10                None
2         GLDG  2024-04-10                None
3         LEDS  2024-04-10                None
4         PCYO  2024-04-10                None
..         ...         ...                 ...
683         WM  2024-04-24  After market close
684        WNC  2024-04-24                None
685       WOLF  2024-04-24                None
686        WSC  2024-04-24                None
687       YHGJ  2024-04-24                None

[688 rows x 3 columns]


In [13]:
print(earnings_release_next_month['act_symbol'])

0       AGX
1      APOG
2      GLDG
3      LEDS
4      PCYO
       ... 
683      WM
684     WNC
685    WOLF
686     WSC
687    YHGJ
Name: act_symbol, Length: 688, dtype: object


In [34]:
tk_0 = take_ticker()
df_date = find_earnings_dates(tk_0)
df_0 = fetch_stock_prices_around_earnings(tk_0, df_date['date'])
df_chg = calculate_percentage_changes(df_0)

Enter a stock ticker:  meta


In [43]:
df_date

,act_symbol,date,when
0,META,2022-07-28,After market close
1,META,2022-10-27,After market close
2,META,2023-02-02,After market close
3,META,2023-04-27,After market close
4,META,2023-07-27,After market close
5,META,2023-10-26,After market close
6,META,2024-02-02,After market close
7,META,2024-04-24,None


In [44]:
df_chg_sorted = df_chg.sort_values(by=['period', 'date'])
df_chg_sorted.head(12)

,date,close,open,high,low,period,close_pct_change,lowest_low,highest_high,vix,largest_pct_diff,first,last,total_close_pct_change
0,2022-07-25,166.65,169.00,170.57,164.81,1,NaN,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
1,2022-07-26,159.15,165.94,166.00,157.95,1,-4.500450,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
2,2022-07-27,169.58,162.59,170.87,162.02,1,6.553566,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
3,2022-07-28,160.72,161.06,161.51,154.85,1,-5.224673,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
4,2022-07-29,159.10,157.69,160.14,155.17,1,-1.007964,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
5,2022-08-01,159.93,157.25,165.19,155.23,1,0.521684,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
6,2022-08-02,160.19,158.31,162.24,158.01,1,0.162571,154.85,170.87,4.237426,10.345496,166.65,160.19,-3.876388
7,2022-10-24,129.72,127.25,133.48,124.57,2,NaN,92.60,138.35,10.904958,49.406048,129.72,95.20,-26.611163
8,2022-10-25,137.51,130.88,138.35,130.59,2,6.005242,92.60,138.35,10.904958,49.406048,129.72,95.20,-26.611163
9,2022-10-26,129.82,131.68,135.55,128.53,2,-5.592321,92.60,138.35,10.904958,49.406048,129.72,95.20,-26.611163


In [19]:

df_summary = df_chg_sorted.groupby('period').agg(
    date=('date', 'first'),  # Get the first date of the period
    largest_pct_diff=('largest_pct_diff', 'first'),  # Assuming these are constant within each period
    total_close_pct_change=('total_close_pct_change', 'first'),  # Ditto
    volatility = ('vix', 'first')
).reset_index()

In [20]:
df_summary

,period,date,largest_pct_diff,total_close_pct_change,volatility
0,1,2022-07-25,10.345496,-3.876388,4.174564
1,2,2022-10-24,49.406048,-26.611163,19.496665
2,3,2023-01-30,34.168084,30.300558,20.749784
3,4,2023-04-24,18.244581,12.430095,16.391350
4,5,2023-07-24,13.146028,10.664929,14.031947
5,6,2023-10-23,13.940587,-4.057196,8.911926
6,7,2024-01-29,25.538621,17.098898,37.091073


In [12]:
from pygooglenews import GoogleNews
from bs4 import BeautifulSoup
import requests


def scrape_article_text(article_url):
    try:
        response = requests.get(article_url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Example: Find the article text within <p> tags. This might need to be adjusted
        paragraphs = soup.find_all('p')
        article_text = ' '.join([para.get_text() for para in paragraphs])
        return article_text
    except Exception as e:
        print(f"Could not scrape {article_url}: {e}")
        return None


gn = GoogleNews(lang = 'en', country = 'US')
search_results = gn.search('supply chain')

articles_data = []
for entry in search_results['entries']:
    article_url = entry['link']
    article_text = scrape_article_text(article_url)
    
    if article_text:
        articles_data.append({
            'date': entry['published'].split('T')[0],
            'title': BeautifulSoup(entry['title'], 'html.parser').get_text(),
            'url': article_url,
            'text': article_text[:500]  # Store first 500 characters for demonstration
        })

df_articles = pd.DataFrame(articles_data)
print(df_articles.head())

# Save the DataFrame to a CSV file
df_articles.to_csv('full_text_supply_chain_articles.csv', index=False)

ModuleNotFoundError: ModuleNotFoundError: No module named 'pygooglenews'